In [15]:
import gradio as gr
gr.close_all()
def increase(state):
    new_value = state + 1
    return  gr.update(value=new_value)  # 更新 UI 和 state

with gr.Blocks() as demo:
    state = gr.State(0)  # 初始化状态变量
    output = gr.Markdown(f"{state.value}")
    button = gr.Button("增加")

    button.click(fn=increase, inputs=state, outputs=[state])  # 更新 state 和 UI

demo.launch()


Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


In [6]:
import gradio as gr

philosophy_quotes = [
    ["I think therefore I am."],
    ["The unexamined life is not worth living."]
]


startup_quotes = [
    ["Ideas are easy. Implementation is hard"],
    ["Make mistakes faster."]
]

def show_startup_quotes():
    return gr.Dataset(samples=startup_quotes)

with gr.Blocks() as demo:
    textbox = gr.Textbox()
    dataset = gr.Dataset(components=[textbox], samples=philosophy_quotes)
    button = gr.Button()

    button.click(show_startup_quotes, None, dataset)

demo.launch()


Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


In [3]:
import gradio as gr

# 示例数据：每行包含 (文件名, 向量)
vectorized_files = [
    ["file1.txt", [0.1, 0.2, 0.3, 0.4]],
    ["file2.txt", [0.5, 0.6, 0.7, 0.8]],
    ["file3.txt", [0.9, 1.0, 1.1, 1.2]],
]


with gr.Blocks() as demo:
    gr.Markdown("## 选择向量化文本")
    dataset = gr.Dataset(components=[gr.Textbox(), gr.Textbox()], samples=vectorized_files, label="向量化数据")
 #  dataset.render()

demo.launch()




Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [5]:
import gradio as gr
with gr.Blocks() as demo:
    with gr.Sidebar():
        gr.Textbox()
        gr.Button()


AttributeError: module 'gradio' has no attribute 'Sidebar'

In [11]:
import gradio as gr
import chromadb
from sentence_transformers import SentenceTransformer
import os
import time

# 初始化 ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="text_files")

# 加载嵌入模型
model = SentenceTransformer("all-MiniLM-L6-v2")

# 存储文件内容
data_folder = "uploaded_texts"
os.makedirs(data_folder, exist_ok=True)

def process_file(files):
    """ 处理上传的文件，进行向量化并存入 ChromaDB """
    for file in files:
        file_path = os.path.join(data_folder, file.name)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        # 计算向量
        vector = model.encode(content).tolist()

        # 存入 ChromaDB
        collection.add(
            ids=[file.name],
            embeddings=[vector],
            metadatas=[{"filename": file.name, "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")}]
        )

    return get_stored_files()

def get_stored_files():
    """ 获取已存入 ChromaDB 的文件信息，返回 Dataset 数据 """
    results = collection.get(include=["metadatas"])
    file_data = [[doc["filename"], doc["timestamp"]] for doc in results["metadatas"]]
    return file_data

def search_similar_file(selected_file):
    """ 基于选定文件进行相似度检索 """
    if not selected_file:
        return "请选择一个文件"

    # 获取该文件的向量
    results = collection.get(ids=[selected_file], include=["embeddings"])
    if not results["embeddings"]:
        return "该文件不存在"

    query_vector = results["embeddings"][0]

    # 执行最近邻搜索
    search_results = collection.query(
        query_embeddings=[query_vector],
        n_results=5
    )

    # 提取相似文件名
    matched_files = [doc["filename"] for doc in search_results["metadatas"][0] if doc["filename"] != selected_file]
    return matched_files or ["未找到相似文件"]

# Gradio 界面
with gr.Blocks() as demo:
    gr.Markdown("## 📄 文本向量化与检索系统")

    with gr.Row():
        with gr.Column():
            file_uploader = gr.File(label="📤 上传文本文件", file_types=[".txt", ".md"], file_count="multiple")
            upload_button = gr.Button("向量化文件")
            refresh_button = gr.Button("🔄 刷新文件列表")
          #  file_dataset = gr.Dataset(components=[gr.Textbox(), gr.Textbox()], label="📋 已向量化的文件", headers=["文件名", "存储时间"])
            file_dataset = gr.Dataset(
    components=[gr.Textbox(), gr.Image()],
    label="📋 已上传文件",
    headers=["文件名", "缩略图"],
             samples=[["1111","1111"]]
)

        with gr.Column():
            selected_file = gr.Textbox(label="📌 选定文件")
            search_button = gr.Button("🔍 搜索相似文件")
            result_text = gr.Textbox(label="🔎 检索结果")


    # 绑定事件
    upload_button.click(process_file, inputs=[file_uploader], outputs=[file_dataset])
    refresh_button.click(get_stored_files,  outputs=[file_dataset])
    file_dataset.select(lambda row: row[0], outputs=[selected_file])  # 选择文件时更新输入框
    search_button.click(search_similar_file, inputs=[selected_file], outputs=[result_text])

demo.launch()


D:\Anaconda\lib\site-packages\gradio\utils.py:1002: UserWarning: Expected 1 arguments for function <function <lambda> at 0x0000021B872A8AF0>, received 0.
  warnings.warn(
D:\Anaconda\lib\site-packages\gradio\utils.py:1006: UserWarning: Expected at least 1 arguments for function <function <lambda> at 0x0000021B872A8AF0>, received 0.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
